# Ensemble des import necessaire

In [1]:
import torch
import seaborn
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertForSequenceClassification, CamembertTokenizer, AdamW

# Parsing du fichier

In [2]:
#Parsing du fichier
res = []

with open("./corpus__fasttext.txt","r", encoding="utf-8") as f:
    for line in f:
        label, sentence = line.split(" ",1)   
        res.append((label.replace("__label__",""), sentence))    
        
df = pd.DataFrame(res,columns=["label","sentence"])
df

,label,sentence
0,radical,"""Êtes-vous prêt à faire rendre des comptes à ..."
1,radical,"""Inapproprié [malade] pour vous ? Normal pour ..."
2,radical,"""Les termes «Cabale» et «État profond» font ré..."
3,radical,"""Le pire, c'est qu'ils adorent Satan et commet..."
4,radical,"""Symboles et logos utilisés par les pédophiles..."
...,...,...
1470,non_radical,"""Les crises économique et terroriste de 2008 e..."
1471,non_radical,"""à la différence de la crise financière de 200..."
1472,non_radical,"""Dans l’instant, la photographie de la situati..."
1473,non_radical,"""Ce recul relativement faible du revenu des mé..."


# Infos sur les donnees

In [4]:
#Petite infos
print('Nombre moyen de mots {0:.0f}.'.format(np.mean(dataset['sentence'].apply(lambda x: len(x.split())))))
print('Nombre max de mots {0:.0f}.'.format(np.mean(dataset['sentence'].apply(lambda x: len(x)))))

Nombre moyen de mots 45.
Nombre max de mots 281.


# Preprocessing

In [3]:
#Separation des dataset pour les refusionner plus tard
df_radical = df.groupby('label').get_group('radical')
df_non_radical = df.groupby('label').get_group('non_radical')

#Permet d'avoir les meme proportion pour le df_training et le df_validation
part1_rad, part2_rad = train_test_split(df_radical, test_size=0.2, random_state=13)
part1_non_rad, part2_non_rad = train_test_split(df_non_radical, test_size=0.2, random_state=13)

df_training =  pd.merge(part1_rad, part1_non_rad, how='outer')
df_validation =  pd.merge(part2_rad, part2_non_rad, how='outer')

print(df_training)
df_validation

            label                                           sentence
0         radical  "Vaccination puis puçage obligatoire comme pou...
1         radical  "Le site ultra-sioniste Dreuz.info, se charge ...
2         radical  "Rejoignez l'association si vous voulez partic...
3         radical  "L’ouvrier noir, malgré les efforts de Trump, ...
4         radical  "Ceux q’u’il faut euthanasier, c’est cette pou...
...           ...                                                ...
1175  non_radical  "Ce narcissisme, si bien décrit par l'historie...
1176  non_radical  "Le modèle dominant fondé sur l’énergie bon ma...
1177  non_radical  "Attention, loin de moi l’idée de demander à q...
1178  non_radical  """VOTRE VOIX ET VOTRE VOTE SONT CE QUI COMPTE...
1179  non_radical  "Selon Missoum Chaoui (aumônier pénitencier), ...

[1180 rows x 2 columns]


,label,sentence
0,radical,"""L’islam, moderne , çan’existe pas ! Il suffit..."
1,radical,"""Passé 20 %, le musulman a déjà proclamé la ch..."
2,radical,"""En pratique, c’est le bourgeois frontiste de ..."
3,radical,"""D’après les commentaires des articles de Ripo..."
4,radical,"""L'empire de la République, nous le ressentons..."
...,...,...
290,non_radical,"""L’originalité technocratique française est de..."
291,non_radical,"""Croit-on sérieusement qu’un peuple qu’on main..."
292,non_radical,"""L'émission Quotidien offre un « temps de cerv..."
293,non_radical,"""Je me suis contenté comme bon nombre d’interv..."


In [4]:
#On convertie radical en 1 et pas radical en 0
training_sentence = df_training['sentence'].values.tolist()
training_label = [1 if x=='radical' else 0 for x in df_training['label']]


validation_sentence = df_validation['sentence'].values.tolist()
validation_label = [1 if x=='radical' else 0 for x in df_validation['label']]

In [13]:
#Chargement du tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

#On encode les sentences pour pouvoir les passer a Camenbert
train_sentence_enco = tokenizer.batch_encode_plus(training_sentence,add_special_tokens=True, max_length=90,padding=True,truncation=True,return_attention_mask = True,return_tensors = 'pt')
val_sentence_enco = tokenizer.batch_encode_plus(validation_sentence,add_special_tokens=True, max_length=90,padding=True,truncation=True,return_attention_mask = True,return_tensors = 'pt')


# On transforme en tenseur les labels
train_label_tensor = torch.tensor(training_label)
val_label_tensor = torch.tensor(validation_label)


train_dataset = TensorDataset(train_sentence_enco['input_ids'],train_sentence_enco['attention_mask'],train_label_tensor)

validation_dataset = TensorDataset(val_sentence_enco['input_ids'],val_sentence_enco['attention_mask'],val_label_tensor)


#Taille des batchs qui vont être utilisés pour l'entraînement.
batch_size = 16
 
# Le dataloader un objet iterable
# RandomSampler permet d'iterer le jeu d'entrainement de façon aleatoire
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size)
 
val_dataloader = DataLoader(
            validation_dataset,
            sampler = SequentialSampler(validation_dataset),
            batch_size = batch_size)

# Mise en place du model

In [6]:
# CamembertForSequenceClassification = Camebert + couche de classification
model = CamembertForSequenceClassification.from_pretrained(
    'camembert-base',
    num_labels = 2)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

In [7]:
#L'optimiser ajuste les poids du modèle lors de l'apprentissage
optimizer = AdamW(model.parameters(),
                  lr = 1e-4 , # Learning Rate faible petit jeu de donnees
                  eps = 1e-8 # Epsilon
                 )
epochs = 2 #nb de fois que le modèle va parcourir l'ensemble de données lors de l'entraînement

C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Entrainement du model

In [11]:
# Def de ou vont etre stocker les tenseurs
device = torch.device("cpu")
 
def train(model,train_dataloader,epochs) :
    # Boucle d'entrainement
    for epoch in range(0, epochs):

        print(f"\nEpoch {epoch+1} / {epochs}")
        print("Runing...")

        # On reinitialise la loss pour cette epoque
        total_train_loss = 0

        # Modele en mode train
        model.train()

        # Pour chaque batch
        for step, batch in enumerate(train_dataloader):

            # Recuperation des donnees du batch
            input_id = batch[0].to(device)
            attention_mask = batch[1].to(device)
            label = batch[2].to(device)

            # On met le gradient a 0
            model.zero_grad()        

            # On passe la donnee au model et on recupere la loss
            md = model(input_id, token_type_ids=None,attention_mask=attention_mask,labels=label)
            loss = md.loss

            # .item() donne la valeur numerique de la loss
            total_train_loss += loss.item()

            if (step % 5 == 0 and not step == 0) or step == len(train_dataloader)-1:
                print(f"\tloss moyen : {total_train_loss/step} etape {step}  sur {len(train_dataloader)}.")

            # Backprop
            loss.backward()

            # On actualise les parametrer grace a l'optimizer
            optimizer.step()

        print("\tMoyenne de la loss: {0:.2f}".format(total_train_loss / len(train_dataloader) ))  

train(model,train_dataloader,epochs)
print("Model sauvegarder!")
torch.save(model.state_dict(), "save.pt")


Epoch 1 / 2
Runing...
	loss moyen : 0.8421237587928772 etape 5  sur 74.
	loss moyen : 0.748769211769104 etape 10  sur 74.
	loss moyen : 0.7255592743555704 etape 15  sur 74.
	loss moyen : 0.7090709865093231 etape 20  sur 74.
	loss moyen : 0.7001602005958557 etape 25  sur 74.
	loss moyen : 0.6830982804298401 etape 30  sur 74.
	loss moyen : 0.6782545345170158 etape 35  sur 74.
	loss moyen : 0.6721162647008896 etape 40  sur 74.
	loss moyen : 0.6653674536281162 etape 45  sur 74.
	loss moyen : 0.6695031797885895 etape 50  sur 74.
	loss moyen : 0.6688793204047463 etape 55  sur 74.
	loss moyen : 0.6692936152219773 etape 60  sur 74.
	loss moyen : 0.6602517247200013 etape 65  sur 74.
	loss moyen : 0.6520907163619996 etape 70  sur 74.
	loss moyen : 0.6515534454829073 etape 73  sur 74.
	Moyenne de la loss: 0.64

Epoch 2 / 2
Runing...
	loss moyen : 0.6018055081367493 etape 5  sur 74.
	loss moyen : 0.5256553888320923 etape 10  sur 74.
	loss moyen : 0.520179812113444 etape 15  sur 74.
	loss moyen : 

# Evaluation du model

In [14]:
def evaluation(dataloader,model) :
    # Mettre le modèle en mode d'évaluation
    model.eval()

    # Variables pour stocker les prédictions et les vraies étiquettes
    predictions, true_labels = [], []

    # Boucle pour prédire chaque batch de données de test
    for batch in dataloader:

        # Récupérer les données du batch
        input_id = batch[0].to(device)
        attention_mask = batch[1].to(device)
        label = batch[2].to(device)

        # Désactiver l'optimisation pour économiser de la mémoire
        with torch.no_grad():

            # Passer les données au modèle pour obtenir la prédiction et les logits
            md = model(input_id, 
                                 token_type_ids=None, 
                                 attention_mask=attention_mask, 
                                 labels=label)
            logits = md.logits

        # Convertir les logits en prédictions de classe
        batch_predictions = torch.argmax(logits, dim=1)

        # Ajouter les prédictions et les vraies étiquettes à la liste correspondante
        predictions.extend(batch_predictions)
        true_labels.extend(label)

    #Calculer la précision du modèle sur les données de test
    accuracy = metrics.accuracy_score(true_labels, predictions)
    print("Precision:", accuracy)
    
evaluation(val_dataloader,model)

Precision: 0.6542372881355932


In [25]:
#Charger un model
model_path = "save.pt"
model_dict = torch.load(model_path)